<h1>Второе задание</h1>

<div style="font-size:1.2em">
<ol>
    <li><input type="checkbox" checked> Найти процент дней, когда минимум случается до максимума</li>
    <li><input type="checkbox"> Найти 90% или 95% порог для min/open и такой же для max/open </li>
    <li><input type="checkbox" checked> Включить в анализ SHIB coin</li>

</ol>
</div>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

<h2>Get Top 11 Cryptos History</h2>

In [2]:
from coinMarketCap import getTopHistory

dfs = getTopHistory(limit=11, days=90)

<h2>Calc Statistics</h2>

In [3]:
statColumns = [
    'avg[(max - min)/(max + min)]',
    'avg[(max - open)/ open ]',
    'avg[(min - open)/ open ]',
    'std[(max - min)/(max + min)]',
    'std[(max - open)/ open ]',
    'std[(min - open)/ open ]',
]
cryptoStat = pd.DataFrame(columns=statColumns)

for coin in dfs:
    df = dfs[coin]
    data = []

    data.append(((df.high - df.low)/(df.high + df.low)).mean())
    data.append(((df.high - df.open)/df.open).mean())
    data.append(((df.low - df.open)/df.open).mean())

    data.append(((df.high - df.low)/(df.high + df.low)).std())
    data.append(((df.high - df.open)/df.open).std())
    data.append(((df.low - df.open)/df.open).std())

    cryptoStat = cryptoStat.append(pd.DataFrame([data],columns=statColumns,index=[coin]))

cryptoStat.T

,BTC,ETH,BNB,ADA,USDT,SOL,XRP,DOT,DOGE,USDC,SHIB
avg[(max - min)/(max + min)],0.024635,0.031567,0.032640,0.037485,0.000593,0.058555,0.039927,0.050617,0.042166,0.000630,0.074904
avg[(max - open)/ open ],0.026246,0.032335,0.034033,0.038946,0.000613,0.071784,0.041313,0.056212,0.045768,0.000590,0.097887
avg[(min - open)/ open ],-0.022994,-0.030621,-0.031073,-0.035946,-0.000573,-0.047821,-0.038130,-0.045451,-0.039077,-0.000669,-0.061076
std[(max - min)/(max + min)],0.012753,0.016937,0.018655,0.023959,0.000242,0.032045,0.026719,0.029561,0.028379,0.000684,0.065565
std[(max - open)/ open ],0.022583,0.026887,0.029313,0.040530,0.000401,0.067708,0.039479,0.051108,0.054441,0.000379,0.151391
std[(min - open)/ open ],0.025469,0.032207,0.034687,0.040793,0.000346,0.041014,0.046754,0.046801,0.042226,0.001361,0.061075


<h2>Calc fraction of days</h2>

In [4]:
statColumns_2 = [
    '[max - min] / open >  5 %',
    '[max - min] / open > 10 %',
    '[max - min] / open > 15 %',
    '[max - min] / open > 20 %',
]
cryptoStat_2 = pd.DataFrame(columns=statColumns_2)

for coin in dfs:
    df = dfs[coin]
    data = []
    c = (df.high - df.low)/df.open
    data.append(round((len(c[c >= 0.05]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.10]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.15]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.20]) / len(c) * 100),1))
    
    cryptoStat_2 = cryptoStat_2.append(pd.DataFrame([data],columns=statColumns_2,index=[coin]))

cryptoStat_2.T

,BTC,ETH,BNB,ADA,USDT,SOL,XRP,DOT,DOGE,USDC,SHIB
[max - min] / open > 5 %,40.4,58.4,56.2,65.2,0.0,92.1,71.9,87.6,70.8,0.0,82.0
[max - min] / open > 10 %,4.5,10.1,14.6,23.6,0.0,50.6,24.7,40.4,28.1,0.0,51.7
[max - min] / open > 15 %,1.1,1.1,1.1,6.7,0.0,22.5,6.7,13.5,6.7,0.0,30.3
[max - min] / open > 20 %,0.0,1.1,1.1,1.1,0.0,13.5,3.4,6.7,3.4,0.0,23.6


percentage of days

<h2>Count of days when min == open AND max == open</h2>

In [5]:
statColumns_3 = [
    'min == open',
    'max == open',
]
cryptoStat_3 = pd.DataFrame(columns=statColumns_3)

for coin in dfs:
    df = dfs[coin]
    data = []
    data.append(len(df[df['low'] == df['open']]))
    data.append(len(df[df['high'] == df['open']]))
    
    cryptoStat_3 = cryptoStat_3.append(pd.DataFrame([data],columns=statColumns_3,index=[coin]))

cryptoStat_3["sum"] = cryptoStat_3.sum(axis=1)
cryptoStat_3.T

,BTC,ETH,BNB,ADA,USDT,SOL,XRP,DOT,DOGE,USDC,SHIB
min == open,1,1,1,0,0,3,2,0,0,0,0
max == open,1,1,0,2,0,1,1,2,0,1,2
sum,2,2,1,2,0,4,3,2,0,1,2


<h2>Fraction of days with Condition: min < open AND max > open</h2>

In [6]:
statColumns_4 = [
    '[max - min] / open >  5 %',
    '[max - min] / open > 10 %',
    '[max - min] / open > 15 %',
    '[max - min] / open > 20 %',
]
cryptoStat_4 = pd.DataFrame(columns=statColumns_4)

for coin in dfs:
    df = dfs[coin]
    total = len(df)
    df = df[(df['low'] < df['open']) & (df['high'] > df['open'])]
    data = []
    c = (df.high - df.low)/df.open
    data.append(round((len(c[c >= 0.05]) / total * 100),1))
    data.append(round((len(c[c >= 0.10]) / total * 100),1))
    data.append(round((len(c[c >= 0.15]) / total * 100),1))
    data.append(round((len(c[c >= 0.20]) / total * 100),1))
    
    cryptoStat_4 = cryptoStat_4.append(pd.DataFrame([data],columns=statColumns_4,index=[coin]))

cryptoStat_4.T


,BTC,ETH,BNB,ADA,USDT,SOL,XRP,DOT,DOGE,USDC,SHIB
[max - min] / open > 5 %,39.3,58.4,55.1,62.9,0.0,87.6,68.5,85.4,70.8,0.0,79.8
[max - min] / open > 10 %,4.5,10.1,14.6,23.6,0.0,49.4,24.7,40.4,28.1,0.0,50.6
[max - min] / open > 15 %,1.1,1.1,1.1,6.7,0.0,21.3,6.7,13.5,6.7,0.0,29.2
[max - min] / open > 20 %,0.0,1.1,1.1,1.1,0.0,13.5,3.4,6.7,3.4,0.0,22.5


<h2>Final Result for Comparison</h2>

In [7]:
pd.DataFrame(
    cryptoStat_2.T.append(cryptoStat_4.T).values,
    columns=pd.Index(list(dfs.keys())),
    index=pd.MultiIndex.from_product([['All days', 'max > open AND min < open'], statColumns_2]))


BTC   ETH   BNB   ADA  \
All days                  [max - min] / open >  5 %  40.4  58.4  56.2  65.2   
                          [max - min] / open > 10 %   4.5  10.1  14.6  23.6   
                          [max - min] / open > 15 %   1.1   1.1   1.1   6.7   
                          [max - min] / open > 20 %   0.0   1.1   1.1   1.1   
max > open AND min < open [max - min] / open >  5 %  39.3  58.4  55.1  62.9   
                          [max - min] / open > 10 %   4.5  10.1  14.6  23.6   
                          [max - min] / open > 15 %   1.1   1.1   1.1   6.7   
                          [max - min] / open > 20 %   0.0   1.1   1.1   1.1   

                                                     USDT   SOL   XRP   DOT  \
All days                  [max - min] / open >  5 %   0.0  92.1  71.9  87.6   
                          [max - min] / open > 10 %   0.0  50.6  24.7  40.4   
                          [max - min] / open > 15 %   0.0  22.5   6.7  13.5   
                          [max - min] / open > 20 %   0.0  13.5   3.4   6.7   
max > open AND min < open [max - min] / open >  5 %   0.0  87.6  68.5  85.4   
                          [max - min] / open > 10 %   0.0  49.4  24.7  40.4   
                          [max - min] / open > 15 %   0.0  21.3   6.7  13.5   
                          [max - min] / open > 20 %   0.0  13.5   3.4   6.7   

                                                     DOGE  USDC  SHIB  
All days                  [max - min] / open >  5 %  70.8   0.0  82.0  
                          [max - min] / open > 10 %  28.1   0.0  51.7  
                          [max - min] / open > 15 %   6.7   0.0  30.3  
                          [max - min] / open > 20 %   3.4   0.0  23.6  
max > open AND min < open [max - min] / open >  5 %  70.8   0.0  79.8  
                          [max - min] / open > 10 %  28.1   0.0  50.6  
                          [max - min] / open > 15 %   6.7   0.0  29.2  
                          [max - min] / open > 20 %   3.4   0.0  22.5

<h2>Percentage of Days when <u>min price time</u> < <u>max price time</u></h2>

In [8]:
from coinMarketCap import getTopHourly

dfh = getTopHourly(11) #top 11 with hourly price detalization for 3 Mounths

In [9]:
pd.concat(dfh,keys=list(dfh.keys()), axis=1).head()

,BTC,ETH,BNB,ADA,USDT,SOL,XRP,DOT,DOGE,USDC,SHIB
,price,price,price,price,price,price,price,price,price,price,price
timestamp,,,,,,,,,,,
2021-08-11 22:00:00,45766.497719,3169.991985,392.100333,1.790437,1.000446,41.787594,1.009535,21.267390,0.264572,0.999740,0.000008
2021-08-11 23:00:00,45593.635931,3164.245046,391.757036,1.798038,1.000535,41.744115,1.012524,21.402422,0.265263,0.999923,0.000008
2021-08-12 00:00:00,45772.589605,3186.222387,397.373670,1.838219,1.000750,42.321125,1.020160,21.660757,0.274083,1.000093,0.000008
2021-08-12 01:00:00,46025.090220,3231.937181,403.709249,1.835730,1.000040,42.841294,1.038976,22.133556,0.284667,0.999561,0.000008
2021-08-12 02:00:00,46202.142342,3234.000634,404.999857,1.864384,1.000054,43.318047,1.035729,22.219584,0.289357,0.999445,0.000008


<h2>Example for SOL</h2>

In [10]:
dd = dfh['SOL'].resample('D')['price'].agg(['idxmin', 'idxmax'])
dd.rename(columns={'idxmin':'time for min price','idxmax':'time for max price'})

,time for min price,time for max price
timestamp,,
2021-08-11,2021-08-11 23:00:00,2021-08-11 22:00:00
2021-08-12,2021-08-12 14:00:00,2021-08-12 02:00:00
2021-08-13,2021-08-13 00:00:00,2021-08-13 23:00:00
2021-08-14,2021-08-14 10:00:00,2021-08-14 07:00:00
2021-08-15,2021-08-15 07:00:00,2021-08-15 23:00:00
...,...,...
2021-11-05,2021-11-05 14:00:00,2021-11-05 01:00:00
2021-11-06,2021-11-06 02:00:00,2021-11-06 23:00:00
2021-11-07,2021-11-07 19:00:00,2021-11-07 05:00:00


In [11]:
print('count: ',len(dd[dd['idxmin'] < dd['idxmax']]))
print('(for 90 days)')

count:  41
(for 90 days)


<h2>For all Coins</h2>

In [12]:
col_name = ['days when min price time < max price time']
minMaxStat = pd.DataFrame(columns=col_name)
for coin in dfh:
    df = dfh[coin]
    dd = df.resample('D')['price'].agg(['idxmin', 'idxmax'])
    count = len(dd[dd['idxmin'] < dd['idxmax']])
    
    minMaxStat = minMaxStat.append(pd.DataFrame([count / len(dd)],columns=col_name,index=[coin]))
    
minMaxStat

,days when min price time < max price time
BTC,0.549451
ETH,0.527473
BNB,0.560440
ADA,0.505495
USDT,0.527473
SOL,0.450549
XRP,0.428571
DOT,0.505495
DOGE,0.461538
USDC,0.582418
